In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

dataset = pd.read_csv("preprocessed_dataset.csv")

# Assuming you have text_messages and department_labels arrays
text_messages = dataset["Message"][:10]
department_labels = dataset["Department"][:10]

# Tokenize text messages
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(text_messages)
text_sequences = tokenizer.texts_to_sequences(text_messages)

# Convert department labels to numerical format (e.g., one-hot encoding)
label_map = {label: index for index, label in enumerate(set(department_labels))}
numerical_labels = [label_map[label] for label in department_labels]

# Split dataset into training and validation sets
train_text, val_text, train_labels, val_labels = train_test_split(text_sequences, numerical_labels, test_size=0.2, random_state=42)

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_text, train_labels))
val_dataset = tf.data.Dataset.from_tensor_slices((val_text, val_labels))

# Batch and shuffle datasets
BATCH_SIZE = 32
train_dataset = train_dataset.shuffle(len(train_text)).batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

# Define your neural network model using TensorFlow's Keras API
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(label_map), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=10, validation_data=val_dataset)


2024-02-20 00:10:50.239741: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Can't convert non-rectangular Python sequence to Tensor.